##Make sure you are using a GPU Runtime

Install dependencies

In [ ]:
!pip install transformers pandas scikit-learn datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


Imports

In [ ]:
import torch
import pandas as pd
import json
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import shuffle
from datasets import load_from_disk
from tqdm import tqdm

Connect to drive

Note: Before running this, add the VIP folder as a shortcut to your drive

*Borrowed this from 4644 lol*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

import os
#change to desired path
os.chdir("./drive/MyDrive/VIP/router")
%pwd

Mounted at /content/drive


'/content/drive/.shortcut-targets-by-id/1UiU6uLtX7kvXdtuDnHtarlFc7ijUrMO6/VIP/router'

Datasets list (from Dataset Downloader)

In [ ]:
flare_datasets = [
    "ChanceFocus/flare-fpb",
    "ChanceFocus/flare-fiqasa",
    "ChanceFocus/flare-headlines",
    "ChanceFocus/flare-finqa",
    "ChanceFocus/flare-convfinqa",
    "ChanceFocus/flare-finer-ord",
    "ChanceFocus/flare-sm-bigdata",
    "ChanceFocus/flare-sm-acl",
    "ChanceFocus/flare-sm-cikm",
    "ChanceFocus/flare-german",
    "ChanceFocus/flare-australian",
    "ChanceFocus/flare-edtsum",
    "ChanceFocus/flare-fomc",
    "ChanceFocus/flare-ner"
]

Load datasets

*tqdm idea/usage borrowed from 4664*

In [ ]:
loaded_datasets = {}
with tqdm(flare_datasets, position = 0, desc = 'Dataset') as tdqm_datasets:
  for flare_dataset in tdqm_datasets:
      loaded_datasets[flare_dataset] = load_from_disk(f"./data/{flare_dataset}")

all_data = []
for dataset_name, dataset in loaded_datasets.items():
    for split in dataset.keys():
        if split in ['train', 'validate', 'test']:
            df = pd.DataFrame(dataset[split]['query'], columns=['query'])
            df['label'] = dataset_name
            all_data.append(df)

combined_df = pd.concat(all_data)

Dataset: 100%|██████████| 14/14 [01:21<00:00,  5.79s/it]


Create label dictionary

In [ ]:
label_unique = combined_df['label'].unique()
label_dict = {label: index for index, label in enumerate(label_unique)}
combined_df['label'] = combined_df['label'].map(label_dict)

with open('label_dict.json', 'w') as ld_file:
    json.dump(label_dict, ld_file)

Shuffle the dataframe

This ensures that locally related rows arent kept together in training/validation to create invalid splits

In [ ]:
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

Get pretrained BERT and setup preprocessing framework

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx]['query'])
        label = int(self.dataframe.iloc[idx]['label'])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )

        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Hyperparameters

In [ ]:
MAX_LEN = 256
BATCH_SIZE = 32
NUM_EPOCHS = 1
LEARNING_RATE = 1e-7

Create data split

In [ ]:
train_df, val_df = train_test_split(combined_df, test_size=0.2)
train_dataset = TextDataset(train_df, tokenizer, MAX_LEN)
val_dataset = TextDataset(val_df, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

Train

*tqdm idea/usage borrowed from 4664*

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_unique))
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

def evaluate(model, val_loader):
    model.eval()
    val_accuracy = []
    val_precision = []
    val_recall = []

    with tqdm(val_loader, position = 0, desc = 'Test Batch') as tdqm_val_loader:
      for batch in tdqm_val_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          with torch.no_grad():
              outputs = model(input_ids, attention_mask=attention_mask)

          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          metrics = precision_recall_fscore_support(labels.cpu(), predictions.cpu(), average='micro')[:3]

          val_accuracy.append(accuracy_score(labels.cpu(), predictions.cpu()))
          val_precision.append(metrics[0])
          val_recall.append(metrics[1])

    avg_acc = sum(val_accuracy) / len(val_accuracy)
    avg_precision = sum(val_precision) / len(val_precision)
    avg_recall = sum(val_recall) / len(val_recall)
    return avg_acc, avg_precision, avg_recall

with tqdm(range(NUM_EPOCHS), position = 1, desc = 'Training Progress') as tdqm_epochs:
  for epoch in tdqm_epochs:
      model.train()
      with tqdm(train_loader, position = 0, desc = 'Train Batch') as tdqm_train_loader:
        for batch in tdqm_train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

      acc, precision, recall = evaluate(model, val_loader)
      print(f"Epoch {epoch + 1} - Accuracy: {acc}, Precision: {precision}, Recall: {recall}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

Test Batch: 100%|██████████| 977/977 [05:45<00:00,  2.83it/s]

Training Progress: 100%|██████████| 1/1 [45:46<00:00, 2746.26s/it]

Epoch 1 - Accuracy: 0.8598601160013648, Precision: 0.8598601160013648, Recall: 0.8598601160013648


Save model

In [ ]:
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

Quick predict - change text at the bottom

Meant to test right after training, for full routing use other notebook

In [ ]:
def predict(text, model, tokenizer, max_len):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    for label, index in label_dict.items():
        if index == prediction:
            return label

predicted_label = predict("Evaluate the sentiment of the following text: Foxconn to Invest $1.5 Billion to Expand India Operations", model, tokenizer, MAX_LEN)
print(f"Predicted label: {predicted_label}")